In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

In [2]:
# dummy inputs
x = torch.tensor(1, requires_grad=True)
w = torch.tensor(2, requires_grad=True)
b = torch.tensor(3, requires_grad=True)

In [3]:
# make forward pass & see the grads
y = w*x +b
y.backward()
x.grad, w.grad, b.grad

(tensor(2), tensor(1), tensor(1))

In [4]:
# another dummy inputs
x = torch.randn(10,3)
y = torch.randn(10,2)

In [5]:
# model
linear = nn.Linear(3,2)
linear.weight, linear.bias, linear.parameters()

(Parameter containing:
 tensor([[-0.0497, -0.3760, -0.3046],
         [-0.5077,  0.1172, -0.0910]]), Parameter containing:
 tensor([-0.1991, -0.3434]), <generator object Module.parameters at 0x10d3162b0>)

In [6]:
# loss func
criterion = nn.MSELoss()

In [7]:
# optimizer # you basically send parameters of the model into optimizer and it'll handle them nice
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [8]:
# foward pass
pred = linear(x)
pred, y

(tensor([[ 0.0790, -0.0934],
         [-0.4460, -0.2057],
         [-0.7174,  0.2380],
         [-0.4797,  0.3544],
         [-0.1732, -0.2266],
         [-0.1937, -0.7903],
         [-0.2570,  0.4832],
         [-0.7350, -0.5442],
         [-0.3645, -0.8070],
         [-0.8398, -0.5036]]), tensor([[-1.4508, -0.8751],
         [-0.1515,  0.8508],
         [-0.7762, -1.1925],
         [ 0.8390,  0.4313],
         [-0.2570, -0.5638],
         [-0.2794, -0.8816],
         [ 0.3251, -2.1437],
         [-1.7458,  0.8672],
         [ 1.1277, -0.3533],
         [-0.0186,  1.7718]]))

In [9]:
# training operations: calculating loss
loss = criterion(pred, y)
loss.backward()
loss.item(), linear.weight.grad, linear.bias.grad

(1.3311126232147217, tensor([[ 0.2061,  0.1099, -0.5320],
         [-0.6442,  0.0488, -0.5805]]), tensor([-0.1740, -0.0006]))

In [10]:
# training operations: modify weights&bias according to grads
optimizer.step()
linear.weight, linear.bias

(Parameter containing:
 tensor([[-0.0517, -0.3771, -0.2993],
         [-0.5013,  0.1167, -0.0852]]), Parameter containing:
 tensor([-0.1973, -0.3434]))

In [11]:
# another training step & clear grad firstly 
optimizer.zero_grad()
pred= linear(x)
loss = criterion(pred, y)
loss.item()
loss.backward()
optimizer.step()

In [12]:
# interchange between torch tensor and numpy
x= np.array([[1,2],[3,4]])
y = torch.from_numpy(x)
z = y.numpy()
x,y,z

(array([[1, 2],
        [3, 4]]), tensor([[ 1,  2],
         [ 3,  4]]), array([[1, 2],
        [3, 4]]))

In [13]:
# this causes downloading if /home/users/you/.torch/models has no corresponding model
resnet = torchvision.models.resnet18(pretrained=True) 

In [14]:
# freeze all params and only finetune last layer (fc)
for params in resnet.parameters(): 
    params.requires_grad = False
    #print(params.shape)
params.shape

torch.Size([1000])

In [15]:
# simply replace the top layer for finetuning (also changed num_classes to 100)
resnet.fc = nn.Linear(resnet.fc.in_features, 100) 

In [16]:
# dummpy inputs and forward pass
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
outputs.size()

torch.Size([64, 100])

In [17]:
torch.save(resnet.state_dict(), 'resnet18_fc_finetune.ckpt')
resnet.load_state_dict(torch.load('resnet18_fc_finetune.ckpt'))